In [1]:
import numpy as np
import cvxpy as cp

In [2]:
cp.installed_solvers()

['CVXOPT', 'ECOS', 'ECOS_BB', 'GLPK', 'GLPK_MI', 'OSQP', 'SCIPY', 'SCS']

In [3]:
cB = np.array([1, 1, 1])
b = np.array([15, 30, 20])

In [4]:
A1 = np.array([10, 0, 0])
A2 = np.array([0, 7, 0])
A3 = np.array([0, 0, 5])

B = np.array([A1, A2, A3])

#Z_hat is initialized with a negative number
Z = -10

In [5]:
#Initial x vector is calculated- RMP 1
x = cp.Variable(shape = 3)

constraints = [x[0] >= 0, x[1] >=0, x[2] >=0,\
              x[0]*A1+x[1]*A2+x[2]*A3 == b]

obj = x[0]+x[1]+x[2]

prob = cp.Problem(cp.Minimize(obj), constraints )

result = prob.solve()
result, x.value

(9.785714285714281, array([1.5       , 4.28571429, 4.        ]))

In [6]:
#A loop started and iterated to compute B, y_hat, new column and Z_hat
while Z < 0:    
    
    y_hat = cB.dot(np.linalg.inv(B))
     
    #new column calculated-Knapsack
    a = cp.Variable(3, integer = True)

    constraints_a = [a[0] >= 0, a[1] >= 0, a[2] >= 0,\
                  a[0]*7+a[1]*11+a[2]*16 <= 80 ]


    obj_a = a[0]*y_hat[0]+a[1]*y_hat[1]+a[2]*y_hat[2]

    prob_a = cp.Problem(cp.Maximize(obj_a), constraints_a)

    result_a=prob_a.solve()
    
    #some matrix manipulation
    A1, A2, A3, A4 = B[:,0].T, B[:,1], B[:,2], a.value 
    
    ##Calculate Z
    
    Z = 1-result_a

    if Z > 0:
        break
    else:
        #Calculate new RMP
        x = cp.Variable(4)

        constraints = [x[0] >= 0, x[1] >=0, x[2] >=0, x[3] >=0,\
                      x[0]*A1+x[1]*A2+x[2]*A3+x[3]*A4 == b]

        obj = x[0]+x[1]+x[2]+x[3]

        prob = cp.Problem(cp.Minimize(obj), constraints )
        result=prob.solve()
        
        
        #some matrix manipulation
        X = x.value
        X = np.where(X < 1E-5, 0, X)
        
        
        A = np.array([A1, A2, A3, A4]).T
        A_test = np.array([X[0]*A1, X[1]*A2, X[2]*A3, X[3]*A4]).T
        
        ind = np.where(np.sum(np.abs(A_test), axis=0)<=1E-5)[0]
        A = np.delete(A, ind, axis=1)
        B = A
        
        
           
       #Prints
        print("Z_hat value")
        print(round(Z,3))      
        print("")
        
        if Z < 0:
            print("Since Z_hat is negative, we continue")
            print("****************************")
        else:
            print("Since Z_hat is => 0, we stop")
            print("****************************")
    
        
        print("")
        print("B matrix")
        print(B)
        print("")
        
        if Z < 0:
            print("inverse(B) matrix")
            print(np.linalg.inv(B))
        print("")
        
        print("y_hat value")
        print(y_hat)
        print("")
        
        
        
        print("")
        print("optimal value")
        print(result)
        print("")
        
        print("")
        print("optimal solution")
        print(np.round(x.value, 4))
        print("")
        

Z_hat value
-0.1

Since Z_hat is negative, we continue
****************************

B matrix
[[ 0.  0. 11.]
 [ 7.  0.  0.]
 [ 0.  5.  0.]]

inverse(B) matrix
[[0.         0.14285714 0.        ]
 [0.         0.         0.2       ]
 [0.09090909 0.         0.        ]]

y_hat value
[0.1        0.14285714 0.2       ]


optimal value
9.64935065092034


optimal solution
[0.     4.2857 4.     1.3636]

Z_hat value
-0.039

Since Z_hat is negative, we continue
****************************

B matrix
[[ 0. 11.  2.]
 [ 0.  0.  6.]
 [ 5.  0.  0.]]

inverse(B) matrix
[[ 0.          0.          0.2       ]
 [ 0.09090909 -0.03030303  0.        ]
 [ 0.          0.16666667  0.        ]]

y_hat value
[0.09090909 0.14285714 0.2       ]


optimal value
9.45454547320117


optimal solution
[0.     4.     0.4545 5.    ]

Z_hat value
-0.018

Since Z_hat is negative, we continue
****************************

B matrix
[[0. 2. 9.]
 [0. 6. 0.]
 [5. 0. 1.]]

inverse(B) matrix
[[-0.02222222  0.00740741  0.2       ]
